<a href="https://colab.research.google.com/github/Michael-Jimenez-C/CieCom2/blob/main/MapaCII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import folium
import numpy as np
import heapq

In [21]:
fc=lambda a: 2*np.arctan2(pow(a,1/2),pow(1-a,1/2))
fa=lambda lat1,long1,lat2,long2: pow(np.sin((lat2-lat1)/2),2)+np.cos(lat1)*np.cos(lat2)*pow(np.sin((long2-long1)/2),2)
fd=lambda A,B: 6378E1*fc(fa(A[0],A[1],B[0],B[1]))

In [212]:
class Nodo:
  def __init__(self,nombre,coordenadas):
    self.nombre=nombre
    self.coordenadas=coordenadas
    self.conexiones=[]
    self.status=0#0 no visitado, 1 visitado
    self.auxdist=np.infty
  
  def conectar(self,other):
    #d=lambda A,B: pow(pow(A[0]-B[0],2)+pow(A[1]-B[1],2),1/2)
    if other not in self.conexiones:
      self.conexiones.append((other,fd(self.coordenadas,other.coordenadas)))
    if self not in other.conexiones:
      other.conexiones.append((self,fd(self.coordenadas,other.coordenadas)))
  

  def distancia(self,nombreNodo):
    for i in self.conexiones:
      if i[0].nombre==nombreNodo:
        return  [i[0],i[1]]
    distancia=np.infty
    d=[]
    for i in self.conexiones:
      if i[0].status==1:
        continue
      self.status=1
      k=i[0].distancia(nombreNodo)
      if k!=None:
        if i[1]+k[1]<distancia:
          distancia=i[1]+k[1]
          d=[i[0]]
          d.extend(([k[0]] if type(k[0])!=list else k[0]))
    self.status=0
    return [d,distancia]

  import heapq

  def dijkstra(self,grafo, inicio, fin):
    # Creamos un diccionario para almacenar la distancia más corta desde el nodo inicial
    # hasta cada uno de los otros nodos en el grafo. Inicialmente, establecemos todas las
    # distancias en infinito, excepto la distancia desde el nodo inicial hasta sí mismo,
    # que es 0.
    distancias = {nodo: float('inf') for nodo in grafo}
    distancias[inicio] = 0
    
    # Creamos un diccionario para almacenar la ruta más corta desde el nodo inicial hasta
    # cada uno de los otros nodos en el grafo. Inicialmente, establecemos todas las rutas
    # en None.
    ruta_mas_corta = {nodo: None for nodo in grafo}
    
    # Creamos un conjunto para almacenar los nodos que aún no han sido visitados.
    nodos_no_visitados = set(grafo)
    
    # Creamos una cola de prioridad para almacenar los nodos por orden de distancia desde
    # el nodo inicial. Inicialmente, solo añadimos el nodo inicial a la cola.
    cola_prioridad = [(0, inicio)]
    
    while nodos_no_visitados and cola_prioridad:
        # Sacamos el nodo más cercano de la cola de prioridad.
        (distancia_actual, nodo_actual) = heapq.heappop(cola_prioridad)
        
        # Si el nodo actual ya ha sido visitado, lo saltamos.
        if distancia_actual > distancias[nodo_actual]:
            continue
            
        # Para cada vecino del nodo actual, calculamos su distancia desde el nodo inicial
        # a través del nodo actual y la comparamos con la distancia más corta conocida.
        # Si la distancia a través del nodo actual es más corta, actualizamos la distancia
        # más corta y la ruta más corta.
        for vecino, peso in grafo[nodo_actual].items():
            distancia = distancia_actual + peso
            if distancia < distancias[vecino]:
                distancias[vecino] = distancia
                ruta_mas_corta[vecino] = nodo_actual
                heapq.heappush(cola_prioridad, (distancia, vecino))
        
        # Marcamos el nodo actual como visitado.
        nodos_no_visitados.remove(nodo_actual)
    
    # Construimos la ruta desde el nodo inicial hasta el nodo final.
    ruta = [fin]
    while ruta_mas_corta[ruta[-1]] is not None:
        ruta.append(ruta_mas_corta[ruta[-1]])
    ruta.reverse()
    
    # Imprimimos la distancia más corta y la ruta desde el nodo inicial hasta el nodo final.
    print(f"La distancia más corta desde {inicio} hasta {fin} es {distancias[fin]}")
    print(f"La ruta más corta es {' -> '.join(ruta)}")



  def __repr__(self):
    return self.nombre

In [213]:
A=Nodo('A',[0,0])
B=Nodo('B',[1,1])
C=Nodo('C',[3,3])
D=Nodo('D',[2,2])
E=Nodo('E',[4,4])
F=Nodo('F',[6,6])
G=Nodo('G',[5,5])
H=Nodo('H',[7,7])


In [214]:
A.conectar(B)
A.conectar(C)
B.conectar(C)
B.conectar(D)
B.conectar(E)
C.conectar(D)
C.conectar(F)
C.conectar(G)
D.conectar(H)
E.conectar(G)
G.conectar(H)

In [215]:
grafo = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1},
    'E': {'B': 1, 'G': 6},
    'F': {'C': 5},
    'G': {'C': 7, 'E': 6, 'H': 2},
    'H': {'D': 3, 'G': 2}
}

In [216]:
F.distancia("E")

[[C, G, E], 186973.34904664024]

In [219]:
F.dijkstra(grafo,'F', 'E')

La distancia más corta desde F hasta E es inf
La ruta más corta es E


#Mapa

In [188]:
puntos={"Bogota":([4.61,-74.082], "Soacha,La calera,Mosquera".split(",")),
        "Soacha":([4.583,-74.217], "Chipaque,Fusagasuga,La Mesa,Bogota".split(",")),
        "Madrid":([4.733,-74.267], "Facatativa,La Mesa,Mosquera,Funza".split(",")),
        "Facatativa":([4.817,-74.367], "Madrid,La Mesa,La Vega,Villeta".split(",")),
        "Fusagasuga":([4.333,-74.35], "Soacha".split(",")),
        "Funza":([4.717,-74.217], "Mosquera,Madrid,Cota".split(",")),
        "Chia":([4.85,-74.05], "Bogota,Cota,Gachancipa,Zipaquira".split(",")),
        "Zipaquira":([5.02836,-73.9992], "Gachancipa,Chia".split(",")),
        "Girardot":([4.3,-74.8], "Tocaima,Fusagasuga".split(",")),
        "Villeta":([5.017,-74.467], "La Vega,Facatativa".split(",")),
        "Macheta":([5.07983,-73.6072], "Gachancipa".split(",")),
        "La Vega":([4.99946,-74.3399], "Villeta,Bogota".split(",")),
        "Tocaima":([4.45903,-74.634], "Girardot,La Mesa".split(",")),
        "Mosquera":([4.70612,-74.2307], "Madrid,Funza,Bogota".split(",")),
        "La Mesa":([4.633,-74.467], "Soacha,Tocaima".split(",")),
        "Cota":([4.817,-74.1], "Chia,Funza".split(",")),
        "Gachancipa":([4.99228,-73.8712], "Macheta,Chia".split(",")),
        "La calera":([4.717,-73.967], "Bogota,Chipaque".split(",")),
        "Choachi":([4.52748,-73.9257], "Chipaque,La calera".split(",")),
        "Chipaque":([4.44294,-74.0445], "Choachi,Bogota".split(","))}

In [189]:
m=folium.Map(location=[4.61,-74.082], zoom_start=10, min_zoom=10)
for i in puntos:
  folium.Marker(
      puntos[i][0], popup=f"<i>{i}</i>"
  ).add_to(m)

for i in puntos:
  c=puntos[i]
  for j in c[1]:
    folium.PolyLine([c[0],puntos[j][0]], tooltip="").add_to(m)

m

In [190]:
nodos={i:Nodo(i,puntos[i][0]) for i in puntos}

In [191]:
for i in puntos:
  c=puntos[i]
  for j in c[1]:
    nodos[i].conectar(nodos[j])

In [192]:
origen="Fusagasuga"
destino="Macheta"
ruta=nodos[origen].distancia(destino)
print(f"ruta{ruta[0]} Distancia{ruta[1]}")


ruta[0].insert(0,nodos[origen])

for j in range(len(ruta[0])-1):
  folium.PolyLine([ruta[0][j].coordenadas,ruta[0][j+1].coordenadas], color="red").add_to(m)
m

ruta[Soacha, Bogota, Chia, Gachancipa, Macheta] Distancia50380.99315697961
